In [1]:
from feature_function import fga_prediction


SAS = {'Chris Paul': 'SAS',"De'Aaron Fox": 'SAS', "Devin Vassell": 'SAS',"Harrison Barnes": 'SAS',
       'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}
OKC = { 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}
player_names = SAS
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"

results = fga_prediction(player_names, date_list, usage_path, player_base_path, defense_base_path, schedule_base_path)


for player, fga_predictions in results.items():
    print(fga_predictions)


D:/nba_player_csv_historic/season_2022-23/all_quarters/Chris Paul_content.csv
D:/nba_defense_history_csv/defense_csv_2022-23/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2023-24/all_quarters/Chris Paul_content.csv
D:/nba_defense_history_csv/defense_csv_2023-24/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2024-25/all_quarters/Chris Paul_content.csv
D:/nba_defense_history_csv/defense_csv_2024-25/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2022-23/all_quarters/De'Aaron Fox_content.csv
D:/nba_defense_history_csv/defense_csv_2022-23/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2023-24/all_quarters/De'Aaron Fox_content.csv
D:/nba_defense_history_csv/defense_csv_2023-24/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2024-25/all_quarters/De'Aaron Fox_content.csv
D:/nba_defense_history_csv/defense_csv_2024-25/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2022-23/all_quarte

In [ ]:
def build_data_path(base_path, **Kwargs):
    # Replace placeholders with actual values
    for key, value in Kwargs.items():
        base_path=base_path.replace(f"{{{key}}}", str(value))
    return base_path

base_path = "/data/{year}/{month}/{day}/file.txt"
kwargs = {'year': 2025, 'month': '02', 'day': '10'}
year = 2025
build_data_path(base_path, year=year)

In [56]:
import pandas as pd

schedule_df = pd.read_csv("D:/nba_scheduled_csv/schedule_csv_2025/OKC_schedule_content.csv")

schedule_df

,home_team,schedule_team,schedule_matchup,DATE,TIME,location
0,OKC,MEM,@ Memphis,2025-02-08,LIVE,away
1,OKC,MIA,vs Miami,2025-02-12,8:00 PM,home
2,OKC,UTA,@ Utah,2025-02-21,9:30 PM,away
3,OKC,MIN,vs Minnesota,2025-02-24,8:00 PM,home
4,OKC,ATL,@ Atlanta,2025-02-28,7:30 PM,away
5,OKC,HOU,vs Houston,2025-03-03,8:00 PM,home
6,OKC,POR,vs Portland,2025-03-07,8:00 PM,home
7,OKC,DEN,vs Denver,2025-03-10,8:00 PM,home
8,OKC,DET,@ Detroit,2025-03-15,7:00 PM,away
9,OKC,PHI,vs Philadelphia,2025-03-19,8:00 PM,home


In [ ]:
def fga_prediction(player_names: dict, date_list: list, usage_path, player_base_path, defense_base_path, schedule_base_path):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list,usage_path,player_base_path, defense_base_path)

    fga_prediction_results = {}

    for player, team in player_names.items():

        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # print(schedule_df)

    for player, df in fga_prediction_data.items():

        from sklearn.model_selection import train_test_split
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.metrics import mean_squared_error
        from sklearn.model_selection import cross_val_score
        from sklearn.linear_model import LinearRegression
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        import numpy as np



        features = ['PACE', 'team_pace', 'USG', 'DefRtg','MIN_x', 'home_away', 'Date_in_Seconds','OffRtg', 'team_offrtg']
        target = 'FGA'


        # Continue with your existing operations
        timestamp = int(pd.Timestamp('2024-12-31').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
        test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp


        X_train = train_data[features]
        y_train = train_data[target]
        X_test = test_data[features]
        y_test = test_data[target]


        model = LinearRegression()
        model.fit(X_train,y_train)

        y_pred = model.predict(X_test)

        ###### predicting Minutes ##########
        alpha = 0.2

        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()

        last_actual = df['MIN_x'].iloc[-1]  # Last known FGA
        last_smoothed = df['EWMA_MIN'].iloc[-1]  # Last smoothed value
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = next_value.round(2)


        ##################################################################

        last_season = df_defense["season_defense"].iloc[-1] 
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, ['TEAM','PACE', 'DefRtg', 'OffRtg']]

        # display(df_for_schedule)


        first_team = schedule_df['schedule_team'].iloc[0]
        # print(first_team)
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]
        # print(schedule_team_result)

        schedule_defrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'DefRtg'].values[0]

        schedule_pace = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'PACE'].values[0]

        schedule_offrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'OffRtg'].values[0]

        # print("dfrtg:",schedule_defrtg)
        # print("pace",schedule_pace)
        # print("offrtg",schedule_offrtg)

        # This is to turn the first date in schedule into seconds
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9

        # This is to turn the home and away games into a 1 or a zero -> away is 1 and anything is zero
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)


        

        # display(schedule_df)





        X_future = pd.DataFrame({
        # 'EWMA_FGA_2': [next_value], 
        'PACE':[schedule_pace], 
        'team_pace':[df['team_pace'].iloc[-1]], 
        'USG':[df['USG'].iloc[-1]],
        'DefRtg':[schedule_defrtg],
        'MIN_x':[next_value], 
        'home_away':[schedule_df['home_away'].iloc[0]],
        'Date_in_Seconds':[schedule_df['Date_in_Seconds'].iloc[0]],
        'OffRtg':[df['OffRtg'].iloc[-1]],
        'team_offrtg': [schedule_offrtg]
        })

        display(X_future)

        future_predictions = model.predict(X_future)



        # print("this is future",future_predictions)

        fga_prediction_results.update({player:future_predictions[0]})

        
    



    return fga_prediction_results

In [15]:
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path
import pandas as pd
import numpy as np
from IPython.display import display


def his_usage_team(player_names: dict, date_list: list, usage_path,player_base_path,defense_base_path):
    current_player_dic = {}

    for player, team in player_names.items():
        current_player_frames =[]

        for date in date_list:
            usage_path =build_data_path(usage_path,date=date)
            usage_data = pd.read_csv(usage_path)

            #merging player and defense dat into one
            merged_data, current_defense_df = his_player_defense_data(player_base_path,defense_base_path,player,date)

            #adding season to usage_data
            usage_data['season'] = date

            #Getting the player usage percentage for usage data and adding to merge
            player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
            merged_data['USG'] = player_usage

            #adding the current player team pace
            team_stat = current_defense_df.loc[current_defense_df['TEAM'] == team, 'PACE'].values[0]
            merged_data["team_pace"] = team_stat

            # adding current player team OffRtg
            team_offrtg = current_defense_df.loc[current_defense_df['TEAM'] == team, 'OffRtg'].values[0]
            merged_data["team_offrtg"] = team_offrtg

            team_poss = current_defense_df.loc[current_defense_df['TEAM'] == team, 'POSS'].values[0]
            merged_data["team_poss"] = team_poss
            
            # Exclude rows where the TEAM column matches the given team
            merged_data = merged_data[merged_data['TEAM'] != team]


            # merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

            # Turn date into seconds
            merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
            merged_data = merged_data.sort_values(by="Date_in_Seconds")


            # Turn Home/Away game into 1 and 0
            merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
            # Dropping duplicates
            merged_data = merged_data.drop_duplicates()
            
            # Append the DataFrame for this date to the player's list
            current_player_frames.append(merged_data)

        # Combine all dates for the current player into one DataFrame
        current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)


    return current_player_dic, current_defense_df


player_names = { 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


def fga_prediction(player_names: dict, date_list: list, usage_path, player_base_path, defense_base_path, schedule_base_path):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)
    fga_prediction_results = {}

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    from sklearn.preprocessing import StandardScaler




    for player, team in player_names.items():
        # Get schedule data for the player's team
        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # Get player-specific prediction data

        df = fga_prediction_data[player]





        
        # Model training and prediction code
        features = ['PACE', 'team_pace', 'USG', 'DefRtg', 'MIN_x', 'home_away', 'Date_in_Seconds', 'OffRtg', 'team_offrtg']
        target = 'FGA'

        timestamp = int(pd.Timestamp('2024-12-31').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]
        test_data = df[df['Date_in_Seconds'] >= timestamp]

        X_train = train_data[features]
        y_train = train_data[target]
        X_test = test_data[features]
        y_test = test_data[target]

        # Initialize Scaler
        scaler = StandardScaler()
        # Transform Data
        scaled_data_x = scaler.fit_transform(X_train)
        scaled_data_y = scaler.fit_transform(y_train.values.reshape(-1, 1))

        X_train = pd.DataFrame(scaled_data_x, columns=X_train.columns)     
        y_train = pd.DataFrame(scaled_data_y, columns=['features'])   

        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)


        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"{player}, MAE: {mae}, RMSE: {rmse}")

        # EWMA calculation for minutes
        alpha = 0.2
        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()
        last_actual = df['MIN_x'].iloc[-1]
        last_smoothed = df['EWMA_MIN'].iloc[-1]
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = round(next_value, 2)

        # Get defensive stats for the scheduled team
        last_season = df_defense["season_defense"].iloc[-1]
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, ['TEAM', 'PACE', 'DefRtg', 'OffRtg']]

        first_team = schedule_df['schedule_team'].iloc[0]
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]
        schedule_defrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'DefRtg'].values[0]
        schedule_pace = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'PACE'].values[0]
        schedule_offrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'OffRtg'].values[0]

        # Convert schedule dates to seconds
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)

        # Create future prediction dataframe
        X_future = pd.DataFrame({
            'PACE': [schedule_pace],
            'team_pace': [df['team_pace'].iloc[-1]],
            'USG': [df['USG'].iloc[-1]],
            'DefRtg': [schedule_defrtg],
            'MIN_x': [next_value],
            'home_away': [schedule_df['home_away'].iloc[0]],
            'Date_in_Seconds': [schedule_df['Date_in_Seconds'].iloc[0]],
            'OffRtg': [df['OffRtg'].iloc[-1]],
            'team_offrtg': [schedule_offrtg]
        })

        future_predictions = model.predict(X_future)
        fga_prediction_results[player] = future_predictions[0]

    return fga_prediction_results

results = fga_prediction(player_names, date_list, usage_path, player_base_path, defense_base_path, schedule_base_path)


for player, fga_predictions in results.items():
    print(fga_predictions)
   

            

            

            

            



            


D:/nba_player_csv_historic/season_2022-23/all_quarters/Alex Caruso_content.csv
D:/nba_defense_history_csv/defense_csv_2022-23/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2023-24/all_quarters/Alex Caruso_content.csv
D:/nba_defense_history_csv/defense_csv_2023-24/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2024-25/all_quarters/Alex Caruso_content.csv
D:/nba_defense_history_csv/defense_csv_2024-25/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2022-23/all_quarters/Isaiah Hartenstein_content.csv
D:/nba_defense_history_csv/defense_csv_2022-23/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2023-24/all_quarters/Isaiah Hartenstein_content.csv
D:/nba_defense_history_csv/defense_csv_2023-24/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2024-25/all_quarters/Isaiah Hartenstein_content.csv
D:/nba_defense_history_csv/defense_csv_2024-25/all_quarter_defense_content.csv
D:/nba_player_csv_historic/seas

In [16]:
import data_functions as data_functions
import pandas as pd
import numpy as np
import importlib
importlib.invalidate_caches()
from data_functions import his_player_defense_data, current_player_defense_data

#"D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
#"D:\nba_defense_csv_current\defense_csv_2024-25\all_quarter_defense_content.csv"
#"D:\nba_scheduled_csv\schedule_csv_2025\ATL_schedule_content.csv"

player_names = {'Shai Gilgeous-Alexander':'OKC', 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC'}
date_list = ["2022-23","2023-24","2024-25"]
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
# usage_data = f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv"



current_player_dic = {}

for player, team in player_names.items():
  current_player_frames = []

  for date in date_list:
    usage_data = pd.read_csv(f"D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv")  

    # use current_player function to to merge datat
    merged_data, current_player_defense = his_player_defense_data(player_base_path,defense_base_path,player,date)

    # Add the season column to usage_data
    usage_data['season'] = date
    
    # Get the player's usage percentage from the usage data
    # (Assumes that there is exactly one matching row)
    player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
    merged_data['USG'] = player_usage


    # adding current player team pace
    team_stat = current_player_defense.loc[current_player_defense['TEAM'] == team, 'PACE'].values[0]
    merged_data["team_pace"] = team_stat

    # display(current_player_defense)

    # adding current player team OffRtg
    team_offrtg = current_player_defense.loc[current_player_defense['TEAM'] == team, 'OffRtg'].values[0]
    merged_data["team_offrtg"] = team_offrtg

    team_poss = current_player_defense.loc[current_player_defense['TEAM']== team, 'POSS'].values[0]
    merged_data["team_poss"] = team_poss

    # Exclude rows where the TEAM column matches the given team
    merged_data = merged_data[merged_data['TEAM'] != team]
    # display(merged_data.head(5))

    # merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

    # Turn date into seconds
    merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
    merged_data = merged_data.sort_values(by="Date_in_Seconds")

    # Turn Home/Away game into 1 and 0
    merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
    # Dropping duplicates
    merged_data = merged_data.drop_duplicates()
    
    # Append the DataFrame for this date to the player's list
    current_player_frames.append(merged_data)

  # Combine all dates for the current player into one DataFrame
  current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)



specific_player = 'Shai Gilgeous-Alexander'
for player, df in current_player_dic.items():

    print(f"\nData for {player}:")
    df['FGA_rolling_3'] = df['FGA'].rolling(window=3).mean()



    alpha = 0.2

    df['EWMA_FGA'] = df['FGA'].ewm(span=(2/(1-alpha)-1), adjust=False).mean()

    df['EWMA_FGA_2'] = df['FGA'].ewm(span=(2/alpha - 1), adjust=False).mean()

    # alpha = 0.2  # Example smoothing factor
    df['Exp_smooth'] = 21  # Initialize column

    for i in range(1, len(df)):
        df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']

    df_act = df

    # display(df_act)

    # display(df)
    df = df.head(149)

    value_fga_list = []
    moving_average_list = []
    next_next_value_list = []
    date_list = []
    
    for value_fga, moving_average, date in zip(df['FGA'],df['EWMA_FGA_2'],df['Date']):
       value_fga_list.append(value_fga)
       moving_average_list.append(moving_average)
       #print("this is actual:",value_fga, "this is last_predicted",moving_average)
       next_next_value = alpha * value_fga + (1 - alpha) * moving_average
       next_next_value_list.append(next_next_value)
       date_list.append(date)
       #print("this is next:",next_next_value)
       dataframe_dic= {'Date':date_list,'Actual_FGA': value_fga_list, 'Moving_average':moving_average_list, 'Next':next_next_value_list}
       dataframe = pd.DataFrame(dataframe_dic)
      #  dataframe['Next'] = dataframe['Next'].shift(1)

    # display(dataframe)


    # Predict the next (11th) value
    last_actual = df['FGA'].iloc[-1]  # Last known FGA
    last_smoothed = df['EWMA_FGA_2'].iloc[-1]  # Last smoothed value
    next_value = alpha * last_actual + (1 - alpha) * last_smoothed


    # Print and add the prediction
    print(f"Predicted value: {next_value}")

    # display(df)

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error



    features = ['PACE', 'team_pace', 'USG', 'DefRtg','MIN_x', 'home_away', 'Date_in_Seconds','OffRtg', 'team_offrtg']
    target = 'FGA'


     # Continue with your existing operations
    timestamp = int(pd.Timestamp('2024-12-31').timestamp())
    train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
    test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp


    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]


    model = LinearRegression()
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    # print(y_pred)

     # Compare predictions with actual points
    predicted_vs_actual = pd.DataFrame({
        "Date": test_data['Date'],  
        "Matchup": test_data['Matchup'],  
        "Predicted Points": y_pred.round(1), 
        "Actual Points": y_test}).reset_index(drop=True)
    
    display(predicted_vs_actual)

    X_future = pd.DataFrame({
    # 'EWMA_FGA_2': [next_value], 
    'PACE':[df['PACE'].iloc[-1]], 
    'team_pace':[df['team_pace'].iloc[-1]], 
    'USG':[df['USG'].iloc[-1]],
    'DefRtg':[df['DefRtg'].iloc[-1]],
    'MIN_x':[30.09], 
    'home_away':[1],
    'Date_in_Seconds':[1738368000],
    'OffRtg':[df['OffRtg'].iloc[-1]],
    'team_offrtg': [df['team_offrtg'].iloc[-1]]
    })
    
    future_predictions = model.predict(X_future)

    print("this is future",future_predictions)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"MAE: {mae}, RMSE: {rmse}")

    import numpy as np

    # Drop NaN values (first row of 'Next' is NaN due to shift)
    rmse_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute RMSE
    rmse = np.sqrt(((rmse_df['Actual_FGA'] - rmse_df['Next']) ** 2).mean())

    print("RMSE:", rmse)


    # Drop NaN values (due to shift)
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Total number of valid comparisons
    total = len(comparison_df)

    # Count occurrences
    higher_count = (comparison_df['Next'] > comparison_df['Actual_FGA']).sum()
    lower_count = (comparison_df['Next'] < comparison_df['Actual_FGA']).sum()
    equal_count = (comparison_df['Next'] == comparison_df['Actual_FGA']).sum()

    # Calculate percentages
    higher_percent = (higher_count / total) * 100
    lower_percent = (lower_count / total) * 100
    equal_percent = (equal_count / total) * 100

    # Print results
    print(f"Percentage of predictions HIGHER than actual: {higher_percent:.2f}%")
    print(f"Percentage of predictions LOWER than actual: {lower_percent:.2f}%")
    print(f"Percentage of predictions EQUAL to actual: {equal_percent:.2f}%")


    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Filter cases where Next is higher than Actual
    higher_cases = comparison_df[comparison_df['Next'] > comparison_df['Actual_FGA']]
    higher_difference_avg = (higher_cases['Next'] - higher_cases['Actual_FGA']).mean()

    # Filter cases where Next is lower than Actual
    lower_cases = comparison_df[comparison_df['Next'] < comparison_df['Actual_FGA']]
    lower_difference_avg = (lower_cases['Actual_FGA'] - lower_cases['Next']).mean()

    # Print results
    print(f"Average difference when prediction is HIGHER: {higher_difference_avg:.2f}")
    print(f"Average difference when prediction is LOWER: {lower_difference_avg:.2f}")

    # Drop NaN values
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute Mean Error (ME)
    mean_error = (comparison_df['Next'] - comparison_df['Actual_FGA']).mean()

    # Print result
    print(f"Mean Error (ME): {mean_error:.2f}")

    

D:/nba_player_csv_historic/season_2022-23/all_quarters/Shai Gilgeous-Alexander_content.csv
D:/nba_defense_history_csv/defense_csv_2022-23/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2023-24/all_quarters/Shai Gilgeous-Alexander_content.csv
D:/nba_defense_history_csv/defense_csv_2023-24/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2024-25/all_quarters/Shai Gilgeous-Alexander_content.csv
D:/nba_defense_history_csv/defense_csv_2024-25/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2022-23/all_quarters/Alex Caruso_content.csv
D:/nba_defense_history_csv/defense_csv_2022-23/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2023-24/all_quarters/Alex Caruso_content.csv
D:/nba_defense_history_csv/defense_csv_2023-24/all_quarter_defense_content.csv
D:/nba_player_csv_historic/season_2024-25/all_quarters/Alex Caruso_content.csv
D:/nba_defense_history_csv/defense_csv_2024-25/all_quarter_defense_content.csv
D:/nba_player_cs

C:\Users\mandy\AppData\Local\Temp\ipykernel_25436\3820781042.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20.200000000000003' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']


,Date,Matchup,Predicted Points,Actual Points
0,12/31/2024,OKC vs. MIN,21.1,23
1,1/2/2025,OKC vs. LAC,19.1,17
2,1/3/2025,OKC vs. NYK,22.8,26
3,1/5/2025,OKC vs. BOS,23.3,23
4,1/8/2025,OKC @ CLE,22.9,27
5,1/10/2025,OKC @ NYK,18.8,21
6,1/12/2025,OKC @ WAS,19.5,17
7,1/14/2025,OKC @ PHI,20.3,15
8,1/16/2025,OKC vs. CLE,19.6,26
9,1/19/2025,OKC vs. BKN,18.8,14


this is future [19.29037452]
MAE: 3.9415198777143052, RMSE: 4.86421082479937
RMSE: 2.7693003111753947
Percentage of predictions HIGHER than actual: 48.99%
Percentage of predictions LOWER than actual: 49.66%
Percentage of predictions EQUAL to actual: 1.34%
Average difference when prediction is HIGHER: 2.31
Average difference when prediction is LOWER: 2.31
Mean Error (ME): -0.01

Data for Alex Caruso:
Predicted value: 5.699376091469293


C:\Users\mandy\AppData\Local\Temp\ipykernel_25436\3820781042.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']


,Date,Matchup,Predicted Points,Actual Points
0,2025-01-14,OKC @ PHI,5.5,4
1,2025-01-16,OKC vs. CLE,6.9,3
2,2025-01-17,OKC @ DAL,6.6,9
3,2025-01-19,OKC vs. BKN,5.0,6
4,2025-01-22,OKC vs. UTA,6.4,7
5,2025-01-23,OKC vs. DAL,6.7,5
6,2025-01-26,OKC @ POR,6.1,6
7,2025-01-29,OKC @ GSW,3.7,0
8,2025-02-05,OKC vs. PHX,6.2,6
9,2025-02-07,OKC vs. TOR,6.0,7


this is future [7.08087681]
MAE: 1.621321681097197, RMSE: 2.0667992640764874
RMSE: 1.7047596409664296
Percentage of predictions HIGHER than actual: 51.56%
Percentage of predictions LOWER than actual: 46.88%
Percentage of predictions EQUAL to actual: 1.56%
Average difference when prediction is HIGHER: 1.23
Average difference when prediction is LOWER: 1.53
Mean Error (ME): -0.08

Data for Isaiah Hartenstein:
Predicted value: 7.667143529388657


C:\Users\mandy\AppData\Local\Temp\ipykernel_25436\3820781042.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']


,Date,Matchup,Predicted Points,Actual Points
0,2024-12-31,OKC vs. MIN,9.6,10
1,2025-01-02,OKC vs. LAC,8.6,9
2,2025-01-03,OKC vs. NYK,9.9,4
3,2025-01-05,OKC vs. BOS,10.2,9
4,2025-01-08,OKC @ CLE,10.1,14
5,2025-01-10,OKC @ NYK,9.4,6
6,2025-01-12,OKC @ WAS,9.0,7
7,2025-01-14,OKC @ PHI,9.0,7
8,2025-01-26,OKC @ POR,9.9,6
9,2025-01-29,OKC @ GSW,9.8,10


this is future [9.87727157]
MAE: 2.385499665141611, RMSE: 2.8498922146321215
RMSE: 1.7775325183918278
Percentage of predictions HIGHER than actual: 53.12%
Percentage of predictions LOWER than actual: 45.31%
Percentage of predictions EQUAL to actual: 1.56%
Average difference when prediction is HIGHER: 1.21
Average difference when prediction is LOWER: 1.74
Mean Error (ME): -0.15
